<a href="https://colab.research.google.com/github/HannaCarla/HannaCarla/blob/main/Imbalanced_MixedModel_cw1_2107.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install mne

In [ ]:
import mne
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from glob import glob
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Conv1D, MaxPooling2D, Flatten, Dense



In [ ]:
iir_params=dict(order=2, ftype='butter')

In [ ]:
dataframes = []

## Processamento dos dados

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

In [ ]:
#read all file
all_files_path=glob('/content/drive/MyDrive/Hanna/Mix_Pacientes/imb_mixed_model_files/new_sample/*.edf')
print((all_files_path))

In [ ]:
all_files_path

In [ ]:
def preprocessing(file_path):
    datax=mne.io.read_raw_edf(file_path,preload=True)
    datax.set_eeg_reference()
    datax.filter(1., None)
    datax.pick_channels(['F3-C3','C3-P3', 'F4-C4','C4-P4' ])
    datax.filter(None, h_freq=40.0, method='iir', iir_params=iir_params, verbose=True)
    epochs=mne.make_fixed_length_epochs(datax,duration=4)
    epochs=epochs.to_data_frame()
    return epochs

In [ ]:
data=[preprocessing(subject) for subject in all_files_path]

In [ ]:
print(len(data))

In [ ]:
df1 = data[0]
df2 = data[1]
df3 = data[2]
df4 = data[3]
df5 = data[4]
df6 = data[5]

In [ ]:
dataframes = [df1, df2, df3, df4, df5, df6]

In [ ]:
#Função organizar dataframes
for i, df in enumerate (dataframes):
  df['condition'] = 0
  df['registro'] = i + 1


In [ ]:
all_df=pd.concat([df1, df2, df3, df4, df5, df6])

In [ ]:
all_df.info()

In [ ]:
# reindex the columns
all_df = all_df.reindex(columns=['time', 'condition', 'epoch', 'registro', 'F3-C3', 'C3-P3', 'F4-C4', 'C4-P4'])

print(all_df)

# Labels

In [ ]:
def seizureepochs (start_time, duration):
  start_time = start_time  # example seizure start time in seconds
  duration = duration  # example seizure duration in seconds
  sampling_rate = 256  # EEG data sampling rate in Hz
  epoch_len = 4  # epoch length in seconds
  overlap = 0  # epoch overlap in seconds
  epoch_len_samples = epoch_len * sampling_rate  # epoch length in data points
  overlap_samples = overlap * sampling_rate  # epoch overlap in data points
  start_sample = int((start_time - overlap) * sampling_rate)  # calculate starting sample index
  end_sample = int((start_time + duration - overlap) * sampling_rate)  # calculate ending sample index
  start_epoch = int(round(start_sample / epoch_len_samples))  # round to nearest epoch boundary
  end_epoch = int(round(end_sample / epoch_len_samples))  # round to nearest epoch boundary
  seizure_epochs = list(range(start_epoch, end_epoch))  # create list of epochs that correspond to seizure
  return seizure_epochs


Registro chb01_26

In [ ]:
registrochb01_26 = seizureepochs(1862, 101)

In [ ]:
registrochb01_26

In [ ]:
all_df.loc[(all_df['epoch']>=465) & (all_df['epoch']<=489) & (all_df['registro']==1), 'condition']=1

In [ ]:
all_df['condition'].value_counts()

Registro chb03_03

In [ ]:
registrochb03_03 =seizureepochs(432, 69)

In [ ]:
registrochb03_03

In [ ]:
all_df.loc[(all_df['epoch']>=107) & (all_df['epoch']<=123) & (all_df['registro']==6), 'condition']=1

In [ ]:
all_df['condition'].value_counts()

Registro chb05_17

In [ ]:
Registro_chb05_17 = seizureepochs(2451, 120)

In [ ]:
Registro_chb05_17

In [ ]:
all_df.loc[(all_df['epoch']>=612) & (all_df['epoch']<=641) & (all_df['registro']==2), 'condition']=1

In [ ]:
all_df['condition'].value_counts()

Registro Chb08_21

In [ ]:
Registro_Chb08_21 = seizureepochs(2083, 264)

In [ ]:
Registro_Chb08_21

In [ ]:
all_df.loc[(all_df['epoch']>=520) & (all_df['epoch']<=585) & (all_df['registro']==5), 'condition']=1

In [ ]:
all_df['condition'].value_counts()

Registro Chb11_99

In [ ]:
Registro_Chb11_99 = seizureepochs(1454, 752)

In [ ]:
Registro_Chb11_99

In [ ]:
all_df.loc[(all_df['epoch']>=363) & (all_df['epoch']<=550) & (all_df['registro']==3), 'condition']=1

In [ ]:
all_df['condition'].value_counts()

Registro Chb02_16

In [ ]:
registrochb02_16 = seizureepochs(130, 82)

In [ ]:
registrochb02_16

In [ ]:
all_df.loc[(all_df['epoch']>=31) & (all_df['epoch']<=51) & (all_df['registro']==4), 'condition']=1

In [ ]:
all_df['condition'].value_counts()

In [ ]:
#verificar se as labels foram inseridas
unique_values = np.unique(all_df['condition'])
print(unique_values)


# Image Transformation




In [ ]:
from tqdm import tqdm

In [ ]:
def split_array(array, chunk_size):
    rows, cols = array.shape
    num_chunks = rows // chunk_size
    chunks = np.split(array[:num_chunks*chunk_size], num_chunks)
    return chunks

In [ ]:
channel_columns_names = all_df.columns[4:]
frame_length = 256

frames = [
    frame for frame in tqdm(split_array(all_df, frame_length))
    if frame['condition'].unique().shape[0] == 1 and frame['epoch'].unique().shape[0] == 1
]

print(f'Generated {len(frames)} frames')

frames_X = np.array([frame[channel_columns_names].values for frame in tqdm(frames) if frame.shape[0] == frame_length])
frames_Y = np.array([frame['condition'].unique()[0] for frame in tqdm(frames) if frame.shape[0] == frame_length])

Train Test Split



In [ ]:
# split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(frames_X, frames_Y, test_size=0.2, random_state=42)

# split the training data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)


In [ ]:
print('X_train', X_train.shape)
print('y_train', y_train.shape)
print('X_test', X_test.shape)
print('y_test', y_test.shape)
print('X_val',X_val.shape)
print('y_val', y_val.shape)

In [ ]:
pd.DataFrame([pd.Series(data).value_counts(normalize=True) for data in (y_train, y_test, y_val)], index=['Train', 'Test', 'Val'])

# Deep Learning

In [ ]:
import numpy as np
from keras.models import Model
from keras.layers import Input, Conv2D, DepthwiseConv2D, BatchNormalization, Activation, AveragePooling2D, Flatten, Dense

# Define the input shape
# num_epochs = 900
# num_channels = 4
# num_samples = 1024

input_shape = (X_train.shape[1], X_train.shape[2], 1)

# Reshape the input data
# X_train_reshaped = X_train.reshape(-1, num_channels, 1)
# X_test_reshaped = X_test.reshape(X_test.shape[0], X_test.shape[1], X_test.shape[2], 1)

X_train_reshaped = X_train.reshape(*X_train.shape, 1)
X_val_reshaped = X_val.reshape(*X_val.shape, 1)
X_test_reshaped = X_test.reshape(*X_test.shape, 1)

In [ ]:
X_train_reshaped.shape

In [ ]:
from tensorflow.keras.regularizers import l2

def EEGNet2(input_shape, num_classes):
    input_layer = Input(shape=input_shape)

    # First Conv2D layer
    conv1 = Conv2D(filters=16, kernel_size=(64, 1), strides=(1, 1), padding='same', data_format='channels_last',
                   kernel_regularizer=l2(0.01))(input_layer)
    conv1 = BatchNormalization()(conv1)
    conv1 = DepthwiseConv2D(kernel_size=(2, 1), strides=(2, 2), depth_multiplier=1, padding='valid')(conv1)
    conv1 = BatchNormalization()(conv1)
    conv1 = Activation('elu')(conv1)

    # Second Conv2D layer
    conv2 = Conv2D(filters=32, kernel_size=(32, 1), strides=(1, 1), padding='same', data_format='channels_last',
                   kernel_regularizer=l2(0.01))(conv1)
    conv2 = BatchNormalization()(conv2)
    conv2 = DepthwiseConv2D(kernel_size=(2, 1), strides=(2, 2), depth_multiplier=2, padding='valid')(conv2)
    conv2 = BatchNormalization()(conv2)
    conv2 = Activation('elu')(conv2)

    # Third Conv2D layer
    conv3 = Conv2D(filters=32, kernel_size=(16, 1), strides=(1, 1), padding='same', data_format='channels_last',
                   kernel_regularizer=l2(0.01))(conv2)
    conv3 = BatchNormalization()(conv3)
    conv3 = DepthwiseConv2D(kernel_size=(2, 1), strides=(2, 2), depth_multiplier=4, padding='valid')(conv3)
    conv3 = BatchNormalization()(conv3)
    conv3 = Activation('elu')(conv3)

    # Average Pooling layer
    avg_pool = AveragePooling2D(pool_size=(4, 1), strides=(1, 1))(conv3)

    # Flatten layer
    flatten = Flatten()(avg_pool)

    # Dense layer with L2 regularization
    dense = Dense(units=num_classes, activation='sigmoid', kernel_regularizer=l2(0.01))(flatten)

    # Create the model
    model = Model(inputs=input_layer, outputs=dense)

    return model


In [ ]:
# model = EEGNet(input_shape=(900, 4, 1024, 1), num_classes=1)
model = EEGNet2(input_shape=(256, 4, 1), num_classes=1)
history = model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
model.summary()

DL+KFold

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from statistics import mean
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from sklearn.metrics import confusion_matrix
import seaborn as sns
from sklearn.metrics import balanced_accuracy_score, accuracy_score, precision_recall_fscore_support

In [ ]:
# Calculate class weights
class_weight = {0: 1, 1: 20}  # Define the class weights manually

In [ ]:
cnn_accu_stratified_nm = []
cnn_precision_0 = []
cnn_precision_1 = []
cnn_recall_0 = []
cnn_recall_1 = []
cnn_f1_0 = []
cnn_f1_1 = []
models = []
loss_values = []
fold_num = 1

skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

for train_index, test_index in skf.split(X_train_reshaped, y_train):

    X_train_fold, X_val_fold = X_train_reshaped[train_index], X_train_reshaped[test_index]
    y_train_fold, y_val_fold = y_train[train_index], y_train[test_index]

    # model = EEGNet(input_shape=(900, 4, 1024, 1), num_classes=1)
    model = EEGNet2(input_shape=(256, 4, 1), num_classes=1)
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

    history = model.fit(X_train_fold, y_train_fold, class_weight=class_weight)
    y_pred = model.predict(X_val_fold)
    y_pred_bin = (y_pred > 0.5).astype(int)

    accuracy = accuracy_score(y_val_fold, y_pred_bin)
    precision, recall, f1, _ = precision_recall_fscore_support(y_val_fold, y_pred_bin, average=None)

    cnn_accu_stratified_nm.append(accuracy)
    cnn_precision_0.append(precision[0])
    cnn_precision_1.append(precision[1])
    cnn_recall_0.append(recall[0])
    cnn_recall_1.append(recall[1])
    cnn_f1_0.append(f1[0])
    cnn_f1_1.append(f1[1])
    models.append(model)
    loss_values.append(history.history['loss'])

    print(f"Fold {fold_num}: Accuracy={accuracy:.4f}, Precision (class 0)={precision[0]:.4f}, Precision (class 1)={precision[1]:.4f}, Recall (class 0)={recall[0]:.4f}, Recall (class 1)={recall[1]:.4f}, F1-score (class 0)={f1[0]:.4f}, F1-score (class 1)={f1[1]:.4f}")
    fold_num += 1

# Create a dictionary to hold the results
results = {
    'Fold': range(1, fold_num),
    'Accuracy': cnn_accu_stratified_nm,
    'Precision (Class 0)': cnn_precision_0,
    'Precision (Class 1)': cnn_precision_1,
    'Recall (Class 0)': cnn_recall_0,
    'Recall (Class 1)': cnn_recall_1,
    'F1-score (Class 0)': cnn_f1_0,
    'F1-score (Class 1)': cnn_f1_1
}
# Convert the dictionary to a DataFrame
df = pd.DataFrame(results)

# Calculate overall statistics
overall_accuracy = df['Accuracy'].mean()
overall_precision_0 = df['Precision (Class 0)'].mean()
overall_precision_1 = df['Precision (Class 1)'].mean()
overall_recall_0 = df['Recall (Class 0)'].mean()
overall_recall_1 = df['Recall (Class 1)'].mean()
overall_f1_0 = df['F1-score (Class 0)'].mean()
overall_f1_1 = df['F1-score (Class 1)'].mean()

# Print the table and overall statistics
print(df)
print('Maximum Accuracy:', df['Accuracy'].max())
print('Minimum Accuracy:', df['Accuracy'].min())
print('Overall Accuracy:', overall_accuracy)
print('Overall Precision (class 0):', overall_precision_0)
print('Overall Precision (class 1):', overall_precision_1)
print('Overall Recall (class 0):', overall_recall_0)
print('Overall Recall (class 1):', overall_recall_1)
print('Overall F1-score (class 0):', overall_f1_0)
print('Overall F1-score (class 1):', overall_f1_1)


In [ ]:
model_fold_01=models[0]
model_fold_02=models[1]
model_fold_03=models[2]
model_fold_04=models[3]
model_fold_05=models[4]
model_fold_06=models[5]
model_fold_07=models[6]
model_fold_08=models[7]
model_fold_09=models[8]
model_fold_10=models[9]

Validation + check overfitting

In [ ]:
# Obtain predictions on the test data
y_pred_test = model_fold_01.predict(X_test_reshaped)
y_pred_test_bin = (y_pred_test > 0.5).astype(int)

# Evaluate performance metrics on the test data
accuracy_test = accuracy_score(y_test, y_pred_test_bin)
precision_test, recall_test, f1_test, _ = precision_recall_fscore_support(y_test, y_pred_test_bin, average=None)

print(f"Test set: Accuracy={accuracy_test:.4f}, Precision (class 0)={precision_test[0]:.4f}, Precision (class 1)={precision_test[1]:.4f}, Recall (class 0)={recall_test[0]:.4f}, Recall (class 1)={recall_test[1]:.4f}, F1-score (class 0)={f1_test[0]:.4f}, F1-score (class 1)={f1_test[1]:.4f}")


In [ ]:
from sklearn.metrics import precision_recall_curve, average_precision_score
import matplotlib.pyplot as plt

# Calculate precision and recall values
precision, recall, _ = precision_recall_curve(y_test, y_pred_test)

# Find indices for positive class (class 1)
positive_class_index = np.where(y_test == 1)[0]

# Calculate average precision
average_precision = average_precision_score(y_test, y_pred_test)

# Plot precision-recall curve for positive class
plt.plot(recall[positive_class_index], precision[positive_class_index], label=f'Avg Precision = {average_precision:.2f})')

# Set the x and y axis limits
plt.xlim([0.0, 1.05])
plt.ylim([0.0, 1.05])

# Set labels and title
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision-Recall Curve')
# Add a diagonal line for reference
plt.plot([1, 0], [0, 1], 'k--')


#Adcionar linhas de grade
plt.grid()

# Show the plot
plt.legend(loc='lower left')
plt.show()


In [ ]:
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt

# Calculate the ROC curve
fpr, tpr, _ = roc_curve(y_test, y_pred_test, pos_label=1)

# Calculate the AUC score
roc_auc = auc(fpr, tpr)

# Plot the ROC curve
plt.plot(fpr, tpr, label=f'ROC Curve (AUC = {roc_auc:.2f})')

# Set labels and title
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve')

# Set the x and y axis limits
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])

#Adcionar linhas de grade
plt.grid()

# Add a diagonal line for reference
plt.plot([0, 1], [0, 1], 'k--')

# Show the legend and plot
plt.legend(loc='lower right')
plt.show()


In [ ]:
from sklearn.metrics import confusion_matrix

In [ ]:
tn, fp, fn, tp = confusion_matrix(y_test, y_pred_test_bin).ravel()

print("True Negative:", tn)
print("False Positive:", fp)
print("False Negative:", fn)
print("True Positive:", tp)

In [ ]:
confusion_matrix(y_test, y_pred_test_bin)

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

In [ ]:
#Update confusion matrix
conf_mat = confusion_matrix(y_test, y_pred_test_bin)
conf_mat_norm = conf_mat / conf_mat.sum(axis=1, keepdims=True)

In [ ]:
from sklearn.metrics import confusion_matrix
import itertools

# define the colormap
cmap = plt.cm.Blues

# plot the confusion matrix
plt.imshow(conf_mat_norm, interpolation='nearest', cmap=cmap)
plt.colorbar()

# add labels to the plot
tick_marks = np.arange(2)
plt.xticks(tick_marks, ['Interictal', 'Ictal'], rotation=45)
plt.yticks(tick_marks, ['Interictal', 'Ictal'])

# add sample counts to the plot
thresh = conf_mat_norm.max() / 2.
for i, j in itertools.product(range(conf_mat_norm.shape[0]), range(conf_mat_norm.shape[1])):
    plt.text(j, i, format(conf_mat_norm[i, j], '.2f'),
             horizontalalignment="center",
             color="white" if conf_mat_norm[i, j] > thresh else "black")

plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix')

# show the plot
plt.show()
